## Selenium needed for web crawling

In [1]:
pip install selenium

     |████████████████████████████████| 904 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Installing drivers for using Selenium

In [2]:
!apt-get update 
!apt install chromium-chromedriver

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


## Exception handling classes 

In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import os
# from state_map import state_map
from os import path
from selenium.common.exceptions import TimeoutException

### Mandatory for pulling data on server

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

### Master Function for pulling data from AGMARKNET website and saving it to a location folder/Var(Commodity)/Var(State)/mynewdata_Var(year)_Var(month).csv

#### If a file already exists, it will not be downloaded again to all dynamic stopping

In [3]:
chrome_location = '/content/chromedriver'

def downloadWholesaleData(commodity,centres,start_year,end_year,months):
        rootfolder = 'folders/'+str(commodity)
        if not os.path.exists(rootfolder):
                print(rootfolder,' does not exist')
                os.makedirs(rootfolder)
        else:
                print(rootfolder,' exists')
        for centre in centres:
                folderPath = rootfolder+'/'+str(centre)
                if not os.path.exists(folderPath):
                        print(folderPath,' not exists')
                        os.makedirs(folderPath)
                else:
                        print(folderPath,' exists')
                print(centre, commodity)
                for year in range(start_year,end_year+1):
                        for month in months:
                                print(year,month,centre,commodity)
                                fileName = "mynewdata"+"_"+str(year)+"_"+str(month)+".csv"
                                filePath = folderPath + "/" + fileName
                                print('file to open',filePath)
                                print(path.exists(filePath))
                                if path.exists(filePath):
                                    print(filePath, " Exist")
                                    continue
                                print("After")
                                #print("TRYING TO DOWNLOAD THIS FILE",fileName, end =" ")                                
                                driver = webdriver.Chrome(chrome_options=chrome_options)
  
                                for t in range(3):
                                        try:
                                                print("inside try, t=", t)
                                                try_left = 3
                                                driver.set_page_load_timeout(10)
                                                while True:
                                                        try:
                                                            print("driver.get")
                                                            driver.get("https://agmarknet.gov.in/PriceAndArrivals/DatewiseCommodityReport.aspx")
                                                            break
                                                        except TimeoutException:
                                                            print("Timeout exveption")
                                                            try_left-=1
                                                            driver.execute_script("window.stop();")
                                                            if try_left<=0:
                                                                print("Timeout loading url")
                                                                break

                                                driver.set_page_load_timeout(600)
                                                #driver.get('http://agmarknet.gov.in/PriceAndArrivals/DatewiseCommodityReport.aspx')
                                                print('YEAR', end=" ")
                                                driver.find_element_by_xpath("//*[@id=\"cphBody_cboYear\"]/option[contains(text(),\""+str(year)+"\")]").click()
                                                driver.implicitly_wait(400)
                                                print('MONTH', end=" ")
                                                flag = False
                                                for k in range(5):
                                                        try:
                                                            driver.find_element_by_xpath("//*[@id=\"cphBody_cboMonth\"]/option[contains(text(),\""+str(month)+"\")]").click()
                                                            driver.implicitly_wait(400)
                                                            flag = True
                                                            break
                                                        except (StaleElementReferenceException) as x:
                                                            k+=1
                                                if(not flag):
                                                        continue
                                                print('STATE', end=" ")
                                                flag = False
                                                for k in range(5):
                                                        try:
                                                                driver.find_element_by_xpath("//*[@id=\"cphBody_cboState\"]/option[contains(text(),\""+str(centre)+"\")]").click()
                                                                driver.implicitly_wait(400)
                                                                flag = True
                                                                break
                                                        except (StaleElementReferenceException) as x:
                                                                k+=1
                                                if(not flag):
                                                        continue
                                                print('COMMODITY', end=" ")
                                                flag = False
                                                for k in range(5):
                                                        try:
                                                            driver.find_element_by_xpath("//*[@id=\"cphBody_cboCommodity\"]/option[contains(text(),\""+str(commodity)+"\")]").click()
                                                            driver.implicitly_wait(400)
                                                            flag = True
                                                            break
                                                        except (StaleElementReferenceException) as x:
                                                            k+=1
                                                if(not flag):
                                                        continue
                                                print('downloading data', end =' ')
                                                driver.find_element_by_xpath("//*[@id=\"cphBody_btnSubmit\"]").click()
                                                table = driver.find_element_by_xpath("//*[@id=\"cphBody_gridRecords\"]")
                                                rows = table.find_elements_by_tag_name("tr")
                                                st = ''
                                                count=0
                                                for row in rows:
                                                        cells = row.find_elements_by_xpath(".//*[local-name(.)='th' or local-name(.)='td']")
                                                        for cell in cells:
                                                            st += cell.text+','
                                                        st+='\n'
                                                myfile= open(filePath,'a')
                                                myfile.write(st)
                                                myfile.close()
                                                print("saved")
                                                print(month+ "completed", end=" ")
                                                driver.close()
                                                break
                                        except(NoSuchElementException,StaleElementReferenceException) as e:
                                                print("Exception")
                                                continue
                                try:
                                        driver.close()
                                except:
                                        print('BROWSER ALREADY CLOSED')

## Function call, all variable instances picked from AGMARKNET website e.g. Gujarat, January, etc.

In [25]:
commodity = "Cotton"
start_year = 2010
end_year = 2020
# centres = state_map[commodity]
centres = ['Tamil Nadu']
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

In [26]:
downloadWholesaleData(commodity,centres,start_year,end_year,months)

folders/Cotton  exists
folders/Cotton/Tamil Nadu  not exists
Tamil Nadu Cotton
2010 January Tamil Nadu Cotton
file to open folders/Cotton/Tamil Nadu/mynewdata_2010_January.csv
False
After


/home/mathuryash5/toolkits/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use options instead of chrome_options


inside try, t= 0
driver.get
YEAR MONTH STATE COMMODITY downloading data saved
Januarycompleted BROWSER ALREADY CLOSED
2010 February Tamil Nadu Cotton
file to open folders/Cotton/Tamil Nadu/mynewdata_2010_February.csv
False
After


/home/mathuryash5/toolkits/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use options instead of chrome_options


inside try, t= 0
driver.get
YEAR MONTH STATE COMMODITY downloading data saved
Februarycompleted BROWSER ALREADY CLOSED
2010 March Tamil Nadu Cotton
file to open folders/Cotton/Tamil Nadu/mynewdata_2010_March.csv
False
After


...

inside try, t= 0
driver.get
YEAR MONTH STATE COMMODITY downloading data saved
Octobercompleted BROWSER ALREADY CLOSED
2020 November Tamil Nadu Cotton
file to open folders/Cotton/Tamil Nadu/mynewdata_2020_November.csv
False
After


/home/mathuryash5/toolkits/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use options instead of chrome_options


inside try, t= 0
driver.get
YEAR MONTH STATE Exception
inside try, t= 1
driver.get
YEAR MONTH STATE Exception
inside try, t= 2
driver.get
YEAR MONTH STATE Exception
2020 December Tamil Nadu Cotton
file to open folders/Cotton/Tamil Nadu/mynewdata_2020_December.csv
False
After


/home/mathuryash5/toolkits/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use options instead of chrome_options


inside try, t= 0
driver.get
YEAR MONTH STATE Exception
inside try, t= 1
driver.get
YEAR MONTH STATE Exception
inside try, t= 2
driver.get
YEAR MONTH STATE Exception


### Concatenating all the created files

In [ ]:
import os
import glob 
import pandas as pd 

final_df = pd.DataFrame()
not_loaded = []

for k in centres:
  my_dir = "/content/folders/Cotton/"+k
  filelist = []
  filesList = []
  os.chdir( my_dir )

  # Step 2: Build up list of files:
  for files in glob.glob("*.csv"):
      fileName, fileExtension = os.path.splitext(files)
      filesList.append(files) #filename with extension

  # Step 3: Build up DataFrame:
  df = pd.DataFrame()
  for i in filesList:
    # print (i)
    try:
      frame = pd.read_csv(i,error_bad_lines=False)
      frame = frame.iloc[:,0:7]
      df = df.append(frame)
    except:
      not_loaded+=[str(i)]
      continue
  final_df = pd.concat([df, final_df])

b'Skipping line 1122: expected 8 fields, saw 9\n'
b'Skipping line 1110: expected 8 fields, saw 9\n'


In [ ]:
final_df.to_csv('/content/GJ_2016_2020_COTTON.csv', index = False)

Saving all the individual files to drive

In [ ]:
# final_save_df = pd.DataFrame()
# not_saved = []

# for k in centres:
#   my_dir = "/content/folders/Cotton/"+k
#   filelist = []
#   filesList = []
#   os.chdir( my_dir )

#   # Step 2: Build up list of files:
#   for files in glob.glob("*.csv"):
#       fileName, fileExtension = os.path.splitext(files)
#       filesList.append(files) #filename with extension

#   # Step 3: Build up DataFrame:
#   save_df = pd.DataFrame()
#   for i in filesList:
#     # print (i)
#     try:
#       save_df = pd.read_csv(i,error_bad_lines=False)
#       save_df = save_df.iloc[:,0:7]
#       save_df.to_csv('/content/drive/My Drive/AGMARKNET/GUJARAT/'+str(i), index = False)
#     except:
#       not_saved+=[str(i)]
#       continue

b'Skipping line 1122: expected 8 fields, saw 9\n'
b'Skipping line 1110: expected 8 fields, saw 9\n'
